# Génération de texte

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import callbacks, layers, backend as K
from tensorflow.keras import Model

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


<hr>

## Fonctions

In [3]:
def create_input_label(datasetBatch):

    def split_input_target(chunk):
        """ from https://www.tensorflow.org/tutorials/text/text_generation """
        input_text = chunk[:-1]
        target_text = chunk[1:]
        return input_text, target_text

    dataset_split = datasetBatch.map(split_input_target)
    return dataset_split

In [4]:
path_to_file = keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Traitements préliminaires sur les données
Le texte utilisé en apprentissage ici sera un texte de Shakespeare
## Observation du vocabulaire
Créer une fonction `get_vocab` qui retourne l’ensemble des caractères (uniques) présent dans le texte.  
Afficher les caractères et le nombre de caractères.

In [5]:
def get_vocab(text):
    return np.unique(list(text))
print(f"Vocab: {get_vocab(text)}\nLength: {len(get_vocab(text))}")

Vocab: ['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']
Length: 65


## Conversion du texte en entiers
Créer une fonction `text2int` qui convertit le texte en une séquence d’entiers. A chaque caractère est associé un entier entre 0 et le nombre de caractères. Pour cela, créer un dictionnaire associant chaque caractère à un entier. La fonction retourne une liste contenant la séquence d’entiers et le dictionnaire permettant la conversion.

Afficher les 45 premiers caractères du texte et les premiers entiers associés

In [6]:
def text2int(text):
    vocab = get_vocab(text)
    mapping = {k:v for v,k in enumerate(vocab)}
    mapped = [mapping[i] for i in list(text)]
    return mapping, mapped

print(f"Text: {repr(text[:45])}\nMap: {text2int(text[:45])[1]}")

Text: 'First Citizen:\nBefore we proceed any further,'
Map: [6, 13, 17, 18, 19, 1, 5, 13, 19, 13, 23, 10, 14, 3, 0, 4, 10, 11, 15, 17, 10, 1, 21, 10, 1, 16, 17, 15, 8, 10, 10, 9, 1, 7, 14, 22, 1, 11, 20, 17, 19, 12, 10, 17, 2]


Construire un dictionnaire réalisant la conversion inverse du dictionnaire précédent : il a pour clé les entiers et pour valeur les caractères associés.

In [7]:
int2text = {v:k for k,v in text2int(text)[0].items()}

## Formatage des données
construire une fonction `create_examples` qui réalise les opérations suivantes:

- Création d’un objet Dataset à partir d’une liste d’entiers donnée en argument.
- Retourne des exemples de longueurs `seq_length`, où `seq_length` est un argument de la fonction valant par défaut 100.

In [8]:
def create_examples(data, seq_length = 100):
    dataset = (
        tf.data.Dataset
        .from_tensor_slices(data)
        .batch(seq_length, drop_remainder = True)
    )
    return dataset

## Création des entrées et labels
La tâche ici sera d’entraîner le réseau à prédire le caractère suivant. La fonction `create_input_label` réalise un découpage de chaque exemple du batch pour construire l’entrée du réseau et son label associé. Ainsi :

- l’entrée est l’ensemble des caractères de la séquence sauf le dernier (le dernier caractère n’est pas une entrée du réseau puisqu’il s’agit du dernier élément à prédire)
- le label est l’ensemble des caractères de la séquence sauf le premier (le premier étant l’entrée initiale du réseau, le premier élément prédit est le second caractère)

Appliquer la fonction `create_input_label` sur la sortie de la question précédente.

In [9]:
mapdata = create_input_label(create_examples(text2int(text)[1]))

En utilisant la méthode `take` applicable sur un objet `Dataset`, afficher pour les 3 premiers exemples les séquences d’entrée et de label.  Il faudra pour cela convertir les tenseurs en numpy de la forme `monTenseur.numpy()`. Utiliser le dictionnaire de conversion approprié pour que ce soit un affichage de phrases en caractères.

In [10]:
for x,y in mapdata.take(3):
    print(f" Input: {repr(''.join(int2text[i] for i in x.numpy()))}")
    print(f" Label: {repr(''.join(int2text[i] for i in y.numpy()))}")
    print("-"*80)

 Input: 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYo'
 Label: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
--------------------------------------------------------------------------------
 Input: ' are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, yo'
 Label: 'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you'
--------------------------------------------------------------------------------
 Input: " know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet u"
 Label: "know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us"
--------------------------------------------------------------------------------


## Création de batchs d’apprentissage
Découper les exemples créés à la question précédente en batch. La fonction `create_batch` prendra en argument le nombre d’exemples dans les batchs qui vaudra par défaut 64.

In [11]:
def create_batch(data, batch_size = 64):
    return data.batch(batch_size, drop_remainder = True)

In [12]:
BATCH_SIZE = 64
VOCAB_SIZE = len(get_vocab(text))
SEQ_LENGTH = 100

# Construction du modèle
## Définition de l’architecture
Créer une fonction `model_rnn` pour construire le modèle suivant de façon fonctionnelle :

- Définir une couche d’entrée dont les entrées sont de dimension (batch_size, seq_length) donnés en argument de la fonction.
- Construire une couche d’Embedding réalisant une projection des données d’entrée endimension 256.
- Construire une couche SimpleRNN de 512 unités prenant en entrée le tenseur issu de la couche d’embedding. Attention, la couche récurrente doit retourner une séquence(une sortie par entrée).
- Construire une couche totalement connectée (Dense) ayant autant d’unités que de labels possibles et prenant en entrée la sortie de la couche récurrente.
- Définir le modèle Keras (Model) issu de l’architecture décrite ci-dessus.

In [13]:
def model_rnn(batch_size, seq_length, vocab_size):
    input_layer = layers.Input(batch_input_shape = (batch_size, seq_length))
    x = layers.Embedding(vocab_size, 256)(input_layer)
    x = layers.SimpleRNN(512, return_sequences = True)(x)
    x = layers.Dense(vocab_size)(x)
    return Model(inputs = input_layer, outputs = x)

In [14]:
with tf.device('/gpu:0'):
    model1 = model_rnn(BATCH_SIZE, SEQ_LENGTH, VOCAB_SIZE)
model1.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(64, 100)]               0         
_________________________________________________________________
embedding (Embedding)        (64, 100, 256)            16640     
_________________________________________________________________
simple_rnn (SimpleRNN)       (64, 100, 512)            393728    
_________________________________________________________________
dense (Dense)                (64, 100, 65)             33345     
Total params: 443,713
Trainable params: 443,713
Non-trainable params: 0
_________________________________________________________________


## Définition de la fonction de coût (loss)
Construire une fonction loss retournant le résultat de l’entropie croisée dans un cadre d’application multiclasse. La fonction doit avoir deux arguments:

- Les labels associés aux observations (i.e. les entiers associés aux caractères)
- Les logits, la sortie du réseau (i.e. les probabilités de chaque caractère)

In [15]:
def loss_function(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits = True)

## Génération du texte
La fonction `generate_text` a pour but de générer du texte à partir d’une séquence donnée en entrée (start_string).  
Ici chaque caractère va être généré de manière itérative (un par un). On va augmenter la séquence d’entrée avec le dernier caractère prédit pour prédire le caractère suivant.

- Convertir la chaîne de caractère d’entrée en nombres et stocker le résultat dans la variable `input_eval`.
- Convertir l’entier prédit en caractère.
- Stocker successivement les différents caractères prédits et retourner la séquence complète.

In [16]:
def generate_text(model, start_string, char2idx, idx2char, num2generate=1000):

    input_eval = [char2idx[x] for x in list(start_string)]
    eval_tensor = tf.expand_dims(input_eval, 0)
    text = []
    for i in range(num2generate):
        pred_tensor = model.predict(eval_tensor)
        pred_tensor = tf.squeeze(pred_tensor, 0)  # suppression de la dimension du batch
        predicted_int = tf.random.categorical(pred_tensor, num_samples=1)[
            -1, 0].numpy()  # conversion du dernier caractère prédit en valeur numpy
        eval_tensor = tf.expand_dims([predicted_int], 0)  # nouvelle entrée pour la future prédiction
        text.append(idx2char[predicted_int])
    return ''.join(text)

## Apprentissage du réseau et prédiction
Compiler le modèle en utilisant l’optimiseur Adam et la fonction de coût (loss) définie précédemment.  
Ecrire une ligne de commande utilisant la fonction `fit` d’un modèle Keras afin de réaliser l’apprentissage du modèle sur 5 epochs.  
Ecrire une ligne de commande récupérant la séquence de prédictions issue de la fonction `generate_text` pour la séquence d’entrée de votre choix.  

In [17]:
with tf.device('/gpu:0'):
    model1.compile(
        loss=loss_function,
        optimizer='adam'
    )

In [18]:
dataset = create_batch(create_input_label(create_examples(text2int(text)[1])))
with tf.device('/gpu:0'):
    hist1 = model1.fit(
        dataset,
        epochs = 5,
        verbose = 1,
        use_multiprocessing = True
    )

Epoch 1/5
174/174 [==============================] - 8s 48ms/step - loss: 2.7711
Epoch 2/5
174/174 [==============================] - 8s 48ms/step - loss: 2.1878
Epoch 3/5
174/174 [==============================] - 8s 48ms/step - loss: 2.0171
Epoch 4/5
174/174 [==============================] - 9s 49ms/step - loss: 1.8938
Epoch 5/5
174/174 [==============================] - 8s 47ms/step - loss: 1.8008


# Améliorer son réseau
- Remplacer la couche SimpleRNN par une couche GRU
- Ajouter une 2ème couche récurrente avant la couche Dense
- Modifier votre fonction `model_rnn` en spécifiant que certaines dimensions ne sont pas précisée (valeur `None`)
- Doubler la taille des couches récurrentes
- Ajouter du dropout dans votre réseau
- Augmenter le nombre d’epochs
- Mettre `stateful = True` sur les couches récurrentes

In [19]:
def model_rnn(embedding, batch_size, vocab_size):
    input_layer = layers.Input(batch_input_shape = (batch_size, None))
    x = layers.Embedding(vocab_size, embedding)(input_layer)
    x = layers.Dropout(.1)(x)
    x = layers.GRU(100, stateful = True, return_sequences = True)(x)
    x = layers.Dropout(.2)(x)
    x = layers.GRU(100, stateful = True, return_sequences = True)(x)
    x = layers.Dropout(.2)(x)
    x = layers.Dense(vocab_size)(x)
    return Model(inputs = input_layer, outputs = x)

In [20]:
with tf.device('/gpu:0'):
    model2 = model_rnn(512, BATCH_SIZE, VOCAB_SIZE)
    model2.compile(
        loss=loss_function,
        optimizer='adam'
    )
    model2.summary()
    hist2 = model2.fit(
        dataset,
        epochs = 5,
        verbose = 1,
        use_multiprocessing = True
    )
    

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(64, None)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (64, None, 512)           33280     
_________________________________________________________________
dropout (Dropout)            (64, None, 512)           0         
_________________________________________________________________
gru (GRU)                    (64, None, 100)           184200    
_________________________________________________________________
dropout_1 (Dropout)          (64, None, 100)           0         
_________________________________________________________________
gru_1 (GRU)                  (64, None, 100)           60600     
_________________________________________________________________
dropout_2 (Dropout)          (64, None, 100)          